In [14]:
# Import libraries
import pandas as pd
import numpy as np
import keras
import time
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import os

Using TensorFlow backend.


In [19]:
step_length = 1    # The step length we take to get our samples from our corpus
epochs = 100       # Number of times we train on our full data
batch_size = 32    # Data samples in each training step
latent_dim = 64    # Size of our LSTM
dropout_rate = 0.2 # Regularization with dropout
model_path = os.path.realpath('./poke_gen_model.h5') # Location for the model
load_model = False # Enable loading model from disk
store_model = True # Store model to disk after training
verbosity = 1      # Print result for each epoch
gen_amount = 10    # How many

In [15]:
result = requests.get("https://pokeapi.co/api/v2/pokemon/?limit=1000").json()
full_names = [result['results'][i]['name'] for i in range(result['count'])]
print(full_names)

['bulbasaur', 'ivysaur', 'venusaur', 'charmander', 'charmeleon', 'charizard', 'squirtle', 'wartortle', 'blastoise', 'caterpie', 'metapod', 'butterfree', 'weedle', 'kakuna', 'beedrill', 'pidgey', 'pidgeotto', 'pidgeot', 'rattata', 'raticate', 'spearow', 'fearow', 'ekans', 'arbok', 'pikachu', 'raichu', 'sandshrew', 'sandslash', 'nidoran-f', 'nidorina', 'nidoqueen', 'nidoran-m', 'nidorino', 'nidoking', 'clefairy', 'clefable', 'vulpix', 'ninetales', 'jigglypuff', 'wigglytuff', 'zubat', 'golbat', 'oddish', 'gloom', 'vileplume', 'paras', 'parasect', 'venonat', 'venomoth', 'diglett', 'dugtrio', 'meowth', 'persian', 'psyduck', 'golduck', 'mankey', 'primeape', 'growlithe', 'arcanine', 'poliwag', 'poliwhirl', 'poliwrath', 'abra', 'kadabra', 'alakazam', 'machop', 'machoke', 'machamp', 'bellsprout', 'weepinbell', 'victreebel', 'tentacool', 'tentacruel', 'geodude', 'graveler', 'golem', 'ponyta', 'rapidash', 'slowpoke', 'slowbro', 'magnemite', 'magneton', 'farfetchd', 'doduo', 'dodrio', 'seel', 'dew

In [16]:
names_duplicates = list(map(lambda s: s.split('-')[0], full_names))
names = list(set(names_duplicates))
names = list(map(lambda s: s + '.', names))
print(names)
input_names = names

['machop.', 'reshiram.', 'solosis.', 'mudbray.', 'slugma.', 'vanillish.', 'porygon2.', 'torterra.', 'forretress.', 'kyogre.', 'arcanine.', 'dustox.', 'buzzwole.', 'noctowl.', 'foongus.', 'magneton.', 'kartana.', 'xurkitree.', 'nihilego.', 'delcatty.', 'elekid.', 'zoroark.', 'heatran.', 'tepig.', 'garbodor.', 'geodude.', 'goldeen.', 'pachirisu.', 'floatzel.', 'zweilous.', 'vikavolt.', 'pyroar.', 'trapinch.', 'cleffa.', 'sudowoodo.', 'hoppip.', 'venipede.', 'phione.', 'moltres.', 'flabebe.', 'claydol.', 'chikorita.', 'oshawott.', 'venomoth.', 'kricketune.', 'crabominable.', 'bounsweet.', 'manaphy.', 'solrock.', 'relicanth.', 'caterpie.', 'graveler.', 'bergmite.', 'golett.', 'doublade.', 'golurk.', 'regirock.', 'crustle.', 'zekrom.', 'exeggutor.', 'swablu.', 'fletchinder.', 'simisear.', 'poochyena.', 'pineco.', 'mismagius.', 'tangrowth.', 'scyther.', 'snorlax.', 'eevee.', 'vullaby.', 'meowstic.', 'ampharos.', 'drampa.', 'clamperl.', 'torkoal.', 'wimpod.', 'carbink.', 'psyduck.', 'grovyle.

In [17]:
# Make it all to a long string
concat_names = '\n'.join(input_names).lower()

# Find all unique characters by using set()
chars = sorted(list(set(concat_names)))
num_chars = len(chars)

# Build translation dictionaries, 'a' -> 0, 0 -> 'a'
char2idx = dict((c, i) for i, c in enumerate(chars))
idx2char = dict((i, c) for i, c in enumerate(chars))

# Use longest name length as our sequence window
max_sequence_length = max([len(name) for name in input_names])

print('Total chars: {}'.format(num_chars))
print('Corpus length:', len(concat_names))
print('Number of names: ', len(input_names))
print('Longest name: ', max_sequence_length)

Total chars: 29
Corpus length: 7559
Number of names:  802
Longest name:  13


In [20]:
sequences = []
next_chars = []

# Loop over our data and extract pairs of sequances and next chars
for i in range(0, len(concat_names) - max_sequence_length, step_length):
    sequences.append(concat_names[i: i + max_sequence_length])
    next_chars.append(concat_names[i + max_sequence_length])

num_sequences = len(sequences)

print('Number of sequences:', num_sequences)
print('First 10 sequences and next chars:')
for i in range(10):
    print('X=[{}]   y=[{}]'.replace('\n', ' ').format(sequences[i], next_chars[i]).replace('\n', ' '))

Number of sequences: 7546
First 10 sequences and next chars:
X=[machop. reshi]   y=[r]
X=[achop. reshir]   y=[a]
X=[chop. reshira]   y=[m]
X=[hop. reshiram]   y=[.]
X=[op. reshiram.]   y=[ ]
X=[p. reshiram. ]   y=[s]
X=[. reshiram. s]   y=[o]
X=[ reshiram. so]   y=[l]
X=[reshiram. sol]   y=[o]
X=[eshiram. solo]   y=[s]


In [21]:
X = np.zeros((num_sequences, max_sequence_length, num_chars), dtype=np.bool)
Y = np.zeros((num_sequences, num_chars), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for j, char in enumerate(sequence):
        X[i, j, char2idx[char]] = 1
    Y[i, char2idx[next_chars[i]]] = 1
    
print('X shape: {}'.format(X.shape))
print('Y shape: {}'.format(Y.shape))

X shape: (7546, 13, 29)
Y shape: (7546, 29)


In [22]:
model = Sequential()
model.add(LSTM(latent_dim, 
               input_shape=(max_sequence_length, num_chars),  
               recurrent_dropout=dropout_rate))
model.add(Dense(units=num_chars, activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

model.summary()





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 64)                24064     
_________________________________________________________________
dense_1 (Dense)              (None, 29)                1885      
Total params: 25,949
Trainable params: 25,949
Non-trainable params: 0
_________________________________________________________________


In [23]:
if load_model:
    model.load(model_path)
else:
    
    start = time.time()
    print('Start training for {} epochs'.format(epochs))
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, verbose=verbosity)
    end = time.time()
    print('Finished training - time elapsed:', (end - start)/60, 'min')
    
if store_model:
    print('Storing model at:', model_path)
    model.save(model_path)

Start training for 100 epochs
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





7546/7546 [==============================] - 19s 2ms/step - loss: 2.5133
Epoch 2/100
7546/7546 [==============================] - 17s 2ms/step - loss: 2.2659
Epoch 3/100
7546/7546 [==============================] - 15s 2ms/step - loss: 2.1950
Epoch 4/100
7546/7546 [==============================] - 16s 2ms/step - loss: 2.1459
Epoch 5/100
7546/7546 [==============================] - 15s 2ms/step - loss: 2.0968
Epoch 6/100
7546/7546 [==============================] - 16s 2ms/step - loss: 2.0475
Epoch 7/100
7546/7546 [==============================] - 17s 2ms/step - loss: 2.0151
Epoch 8/100
7546/7546 [==============================] - 23s 3ms/step - loss: 1.9713
Epoch 9/100
7546/7546 [==============================] - 21s 3ms/step - loss: 1.9362
Epoch 10/100
7546/7546 [==============================] - 19s 3ms/step - loss: 1.9033
Epoch 11/100
7546

Epoch 70/100
7546/7546 [==============================] - 8s 1ms/step - loss: 1.3449
Epoch 71/100
7546/7546 [==============================] - 5s 697us/step - loss: 1.3353
Epoch 72/100
7546/7546 [==============================] - 7s 938us/step - loss: 1.3237
Epoch 73/100
7546/7546 [==============================] - 8s 1ms/step - loss: 1.3222
Epoch 74/100
7546/7546 [==============================] - 6s 743us/step - loss: 1.3327
Epoch 75/100
7546/7546 [==============================] - 8s 1ms/step - loss: 1.3228
Epoch 76/100
7546/7546 [==============================] - 6s 753us/step - loss: 1.3221
Epoch 77/100
7546/7546 [==============================] - 5s 725us/step - loss: 1.3097
Epoch 78/100
7546/7546 [==============================] - ETA: 0s - loss: 1.299 - 9s 1ms/step - loss: 1.2991
Epoch 79/100
7546/7546 [==============================] - 5s 728us/step - loss: 1.2904
Epoch 80/100
7546/7546 [==============================] - 7s 991us/step - loss: 1.3033
Epoch 81/100
7546/7546 [===

In [24]:
# Start sequence generation from end of the input sequence
sequence = concat_names[-(max_sequence_length - 1):] + '\n'

new_names = []

print('{} new names are being generated'.format(gen_amount))

while len(new_names) < gen_amount:
    
    # Vectorize sequence for prediction
    x = np.zeros((1, max_sequence_length, num_chars))
    for i, char in enumerate(sequence):
        x[0, i, char2idx[char]] = 1

    # Sample next char from predicted probabilities
    probs = model.predict(x, verbose=0)[0]
    probs /= probs.sum()
    next_idx = np.random.choice(len(probs), p=probs)   
    next_char = idx2char[next_idx]   
    sequence = sequence[1:] + next_char

    # New line means we have a new name
    if next_char == '\n':

        gen_name = [name for name in sequence.split('\n')][1]

        # Never start name with two identical chars, could probably also
        if len(gen_name) > 2 and gen_name[0] == gen_name[1]:
            gen_name = gen_name[1:]

        # Discard all names that are too short
        if len(gen_name) > 2:
            
            # Only allow new and unique names
            if gen_name not in input_names + new_names:
                new_names.append(gen_name.capitalize())

        if 0 == (len(new_names) % (gen_amount/ 10)):
            print('Generated {}'.format(len(new_names)))

10 new names are being generated
Generated 0
Generated 1
Generated 2
Generated 2
Generated 3
Generated 4
Generated 5
Generated 6
Generated 7
Generated 8
Generated 9
Generated 10


In [25]:
print_first_n = min(10, gen_amount)

print('First {} generated names:'.format(print_first_n))
for name in new_names[:print_first_n]:
    print(name)

First 10 generated names:
Arunchur.
Blaking.
Pigon.
Venopudre.
Shellasd.
Xirhoigs.
Slurvaqua.
Viveler.
Amasdus.
Mawshick.


In [27]:

concat_output = '\n'.join(sorted(new_names))
output_path = os.path.realpath('./generated_names.txt')

with open(output_path, 'w') as f:
    f.write(concat_output)